In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Generate how likely a name is to be a given vs surname

In [ ]:
import pandas as pd
import s3fs

from src.data.filesystem import fopen
from src.data.normalize import normalize_freq_names

In [ ]:
# in
given_freq_path="s3://familysearch-names/processed/tree-preferred-given-aggr.csv.gz"
surname_freq_path="s3://familysearch-names/processed/tree-preferred-surname-aggr.csv.gz"
# out
given_surname_freq_path="s3://familysearch-names/processed/tree-preferred-given-surname-freq.csv.gz"

In [ ]:
given_freq_df = pd.read_csv(given_freq_path, na_filter=False)
given_freq = normalize_freq_names(given_freq_df, is_surname=False, add_padding=False)
given_freq_df = None

In [ ]:
surname_freq_df = pd.read_csv(surname_freq_path, na_filter=False)
surname_freq = normalize_freq_names(surname_freq_df, is_surname=True, add_padding=False)
surname_freq_df = None

In [ ]:
all_names = set(given_freq.keys()).union(surname_freq.keys())
print(len(all_names))

In [ ]:
name_freq = []
for name in all_names:
    name_freq.append({
        "name": name, 
        "given_freq": given_freq.get(name, 0),
        "surname_freq": surname_freq.get(name, 0),
    })

In [ ]:
name_freq_df = pd.DataFrame(name_freq)
print(len(name_freq_df))
name_freq_df.head(5)

In [ ]:
# remove names occurring < 40 times
name_freq_df = name_freq_df[name_freq_df["given_freq"] + name_freq_df["surname_freq"] >= 40]
print(len(name_freq_df))
name_freq_df.head(5)

In [ ]:
name_freq_df["total_freq"] = name_freq_df["given_freq"] + name_freq_df["surname_freq"]
name_freq_df["given_ratio"] = name_freq_df["given_freq"] / name_freq_df["total_freq"]

In [ ]:
name_freq_df.head(5)

In [ ]:
name_freq_df[name_freq_df["total_freq"] < 1000].hist(column="total_freq", bins=100)

In [ ]:
name_freq_df[(name_freq_df["given_ratio"] > 0) & (name_freq_df["given_ratio"] < 1)].hist(column="given_ratio", bins=100)

In [ ]:
ambig_df = name_freq_df[(name_freq_df["given_ratio"] > 0.4) & (name_freq_df["given_ratio"] < 0.8)]
print(len(ambig_df))
ambig_df.head(30)

In [ ]:
# save to csv
name_freq_df.to_csv(given_surname_freq_path, index=False)